# Amazon SageMaker + WhyLabs

In [19]:
import os
import json
import pandas as pd
from joblib import dump
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import boto3
from dotenv import dotenv_values
from utils import delete_model, delete_endpoint_config, delete_endpoint, is_endpoint_running

## AWS configuration

In [6]:
AWS_PROFILE_NAME = "mfa"
session = boto3.session.Session(profile_name=AWS_PROFILE_NAME)
AWS_REGION_NAME = session.region_name

In [7]:
sts = session.client("sts")
sm = session.client('sagemaker', region_name=AWS_REGION_NAME)
AWS_ACCOUNT_ID = sts.get_caller_identity().get("Account")
DOCKER_IMAGE_NAME = "whylabs-sagemaker"

## Download Iris dataset

In [18]:
!cd code/ && chmod +x download_iris.sh && ./download_iris.sh

  0%|                                               | 0.00/3.60k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 3.60k/3.60k [00:00<00:00, 3.02MB/s]
Archive:  iris.zip
  inflating: Iris.csv                
  inflating: database.sqlite         


## Train data

In [21]:
data = pd.read_csv('code/dataset/iris.csv')
# Separating the independent variables from dependent variables
X = data.drop(['Id', 'Species'], axis=1)
y = data['Species']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [23]:
# Train a classifier
print("Train started...")
model = SVC()
model.fit(x_train, y_train)
print("Train ended...")
# Save the model
dump(model, 'code/model.joblib')
print("Model saved!")

Train started...
Train ended...
Model saved!


## Custom image building and pushing to ECR

In [27]:
os.system(f"./build_push.sh {DOCKER_IMAGE_NAME} {AWS_PROFILE_NAME}")

Image name whylabs-sagemaker
Profile name mfa
{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:377983720232:repository/whylabs-sagemaker",
            "registryId": "377983720232",
            "repositoryName": "whylabs-sagemaker",
            "repositoryUri": "377983720232.dkr.ecr.us-east-1.amazonaws.com/whylabs-sagemaker",
            "createdAt": "2021-09-08T17:32:14-05:00",
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            },
            "encryptionConfiguration": {
                "encryptionType": "AES256"
            }
        }
    ]
}
Login Succeeded


#1 [internal] load build definition from Dockerfile
#1 sha256:cc4c7e8f914b38f171550b8a1471ed97c5b57dd3a4393b87b077329d1f536d0f
#1 transferring dockerfile: 37B 0.0s done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:099bd16ad98efdc6b980f0a99439bbdd3b15e6c185a04c4f90b86415e9800abc
#2 transferring context: 34B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 ...

#4 [auth] library/ubuntu:pull token for registry-1.docker.io
#4 sha256:47b3f140945da5d6f0fc00cd51298a2b1bcaf71fd5a4d2d073f7a30acee4664b
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 DONE 1.6s

#5 [ 1/11] FROM docker.io/library/ubuntu:18.04@sha256:9bc830af2bef73276515a29aa896eedfa7bdf4bdbc5c1063b4c457a4bbb8cd79
#5 sha256:d1750e31869fe5a60e2fad31896f5d8b06a6c26d3a20b7f5836401e641279689
#5 DONE 0.0s

#8 [i

The push refers to repository [377983720232.dkr.ecr.us-east-1.amazonaws.com/whylabs-sagemaker]
5f70bf18a086: Preparing
4d451d2c4725: Preparing
71766944aaf8: Preparing
69960a2b0055: Preparing
eb2a7046ea56: Preparing
fbee2ef86e65: Preparing
8cc0a0dc03b0: Preparing
5a8d62feae95: Preparing
d20859943999: Preparing
cb8615e15e41: Preparing
6babb56be259: Preparing
cb8615e15e41: Waiting
6babb56be259: Waiting
d20859943999: Waiting
8cc0a0dc03b0: Waiting
5a8d62feae95: Waiting
fbee2ef86e65: Waiting
5f70bf18a086: Layer already exists
69960a2b0055: Pushed
71766944aaf8: Pushed
eb2a7046ea56: Pushed
4d451d2c4725: Pushed
d20859943999: Layer already exists
fbee2ef86e65: Pushed
cb8615e15e41: Layer already exists
6babb56be259: Layer already exists
5a8d62feae95: Pushed
8cc0a0dc03b0: Pushed
latest: digest: sha256:89393ee182b462861c32a95b32c10ff805d38afc5e29971faa6e58cfbfe47c4d size: 2616


0

## Create SageMaker Endpoint

The steps to deploy a SageMaker model are:

1. Create a model
2. Create an endpoint configuration
3. Create a SageMaker endpoint

### 1. Model Creation

In [28]:
ECR_IMAGE_URI = f"{AWS_ACCOUNT_ID}.dkr.ecr.{AWS_REGION_NAME}.amazonaws.com/{DOCKER_IMAGE_NAME}:latest"
ENDPOINT_NAME = "whylabs-sagemaker"
EXECUTION_ROLE_ARN = f"arn:aws:iam::{AWS_ACCOUNT_ID}:role/SageMakerExecution"
INSTANCE_TYPE = "ml.m4.xlarge"

Load variables important for __WhyLabs configuration__ defined inside __.env file__ as dictionary. This values will be settled once the docker container is running within SageMaker.

In [29]:
# Load .env file as dictionary
environment = dotenv_values("code/.env")

In [30]:
# ECR image to be used
PRIMARY_CONTAINER = {
    'Image': ECR_IMAGE_URI, 
    "Environment": environment,
}

In [31]:
try:
    # Create sagemaker model
    r = sm.create_model(
        ModelName=ENDPOINT_NAME,
        ExecutionRoleArn=EXECUTION_ROLE_ARN,
        PrimaryContainer=PRIMARY_CONTAINER,
    )
    print("SageMaker model created.")
except Exception as e:
    print(e.response["Error"])

SageMaker model created.


### 2. Endpoint Config creation

In [32]:
ENDPOINT_CONFIG_NAME = ENDPOINT_NAME + '-config'

In [33]:
try:
    # create endpoint configuration
    _ = sm.create_endpoint_config(
        EndpointConfigName=ENDPOINT_CONFIG_NAME,
        ProductionVariants=[
            {
                'InstanceType': INSTANCE_TYPE,
                'InitialVariantWeight': 1,
                'InitialInstanceCount': 1,
                'ModelName': ENDPOINT_NAME,
                'VariantName': 'AllTraffic'
            }
        ]
    )
    print("Endpoint configuration created.")
except Exception as e:
    print(e.response["Error"])

Endpoint configuration created.


### 3. Endpoint creation

In [35]:
try:
    # create endpoint
    r = sm.create_endpoint(
        EndpointName=ENDPOINT_NAME,
        EndpointConfigName=ENDPOINT_CONFIG_NAME
    )
    print(f"Completed {ENDPOINT_NAME} model endpoint deployment !!!")
except Exception as e:
    print(e.response["Error"])

{'Message': 'Cannot create already existing endpoint "arn:aws:sagemaker:us-east-1:377983720232:endpoint/whylabs-sagemaker".', 'Code': 'ValidationException'}


## Test Endpoint 

In [36]:
# Payload for /invocations endpoint
payload = json.dumps({
    "sepal_length_cm": 5.1,
    "sepal_width_cm": 3.5,
    "petal_length_cm": 1.4,
    "petal_width_cm": 0.2
})

In [38]:
# Invoke the endpoint using
sg = session.client("runtime.sagemaker", region_name=AWS_REGION_NAME)
status = is_endpoint_running(ENDPOINT_NAME, AWS_PROFILE_NAME, AWS_REGION_NAME)
# Check if model was created successfully
if status == "InService":
    response = sg.invoke_endpoint(
        EndpointName=ENDPOINT_NAME,
        Body=payload,
        ContentType='application/json',
    )
    # Decode the response
    print(json.loads(response["Body"].read().decode("utf-8")))
else:
    print(f"Endpoint status is {status}.")

Endpoint status is Creating


## Delete AWS resources

In [ ]:
status = is_endpoint_running(ENDPOINT_NAME, AWS_PROFILE_NAME, AWS_REGION_NAME)

In [ ]:
if status == "InService":
    delete_model(sm, ENDPOINT_NAME)
    delete_endpoint_config(sm, ENDPOINT_CONFIG_NAME)
    delete_endpoint(sm, ENDPOINT_NAME)